# working on a pretrained model

## pretrained models as feature extractors

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.preprocessing import image
import numpy as np

###### https://medium.com/@kenneth.ca95/a-guide-to-transfer-learning-with-keras-using-resnet50-a81a4a28084b
### Transfer learning for CIFAR using resnet50 of 10 classes

In [ ]:
import tensorflow as tf
import tensorflow.keras as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
def preprocess_data(X, Y):
    """trains a convolutional neural network to classify the dataset"""
    X_p = K.applications.efficientnet.preprocess_input(X)
    Y_p = K.utils.to_categorical(Y, 10)
    return X_p, Y_p

In [ ]:
(trainX, trainy), (testX, testy) = K.datasets.cifar10.load_data()
trainX, trainy = preprocess_data(trainX, trainy)
testX, testy = preprocess_data(testX, testy)

170498071/170498071 [==============================] - 14s 0us/step


In [ ]:
print('x_train shape:', trainX.shape)
print(trainX.shape[0], 'train samples')
print(testX.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [ ]:
inputs = K.Input(shape=(224, 224, 3))
#Loading the EfficientNetB0 model with pre-trained ImageNet weights
ent = K.applications.EfficientNetB0(weights='imagenet',include_top=False,input_tensor=inputs)

16705208/16705208 [==============================] - 2s 0us/step


In [ ]:
for layer in ent.layers:
    layer.trainable = False

In [ ]:
model = K.models.Sequential()
model.add(K.layers.Lambda(lambda x: tf.image.resize(x,(224, 224))))
model.add(ent)

model.add(K.layers.Flatten())

model.add(K.layers.Dense(512, activation='relu'))
model.add(K.layers.Dropout(0.5))
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(10, activation='softmax'))

In [ ]:
model.compile(loss='binary_crossentropy',
     optimizer=K.optimizers.RMSprop(lr=0.0001), metrics=['accuracy'])
model.fit(trainX, trainy, batch_size=32, epochs=10, verbose=1,
     validation_data=(testX, testy), 
     shuffle=True)
model.summary()
# model.save("cifar10.h5")
# best accuarcy at epoch 9 after that overfitting 

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/10
1563/1563 [==============================] - 169s 99ms/step - loss: 0.2732 - accuracy: 0.8474 - val_loss: 0.0641 - val_accuracy: 0.9030
Epoch 2/10
1563/1563 [==============================] - 173s 111ms/step - loss: 0.0654 - accuracy: 0.9028 - val_loss: 0.0496 - val_accuracy: 0.9144
Epoch 3/10
1563/1563 [==============================] - 152s 97ms/step - loss: 0.0492 - accuracy: 0.9256 - val_loss: 0.0470 - val_accuracy: 0.9180
Epoch 4/10
1563/1563 [==============================] - 152s 97ms/step - loss: 0.0401 - accuracy: 0.9409 - val_loss: 0.0455 - val_accuracy: 0.9211
Epoch 5/10
1563/1563 [==============================] - 152s 97ms/step - loss: 0.0335 - accuracy: 0.9525 - val_loss: 0.0451 - val_accuracy: 0.9233
Epoch 6/10
1563/1563 [==============================] - 151s 97ms/step - loss: 0.0297 - accuracy: 0.9589 - val_loss: 0.0474 - val_accuracy: 0.9233
Epoch 7/10
1563/1563 [==============================] - 151s 97ms/step - loss: 0.0260 - accuracy: 0.9662 - val_loss: 

In [ ]:
image_pred=model.predict(testX)
np.argmax(image_pred,axis=1)

313/313 [==============================] - 22s 67ms/step


array([3, 8, 8, ..., 5, 1, 7])

In [ ]:
model = K.models.Sequential()
model.add(K.layers.Lambda(lambda x: tf.image.resize(x,(224, 224))))
model.add(ent)

model.add(K.layers.GlobalAveragePooling2D())

model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(512, activation='relu'))
model.add(K.layers.Dropout(0.5))
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(10, activation='softmax'))

In [ ]:
model.compile(loss='binary_crossentropy',
     optimizer=K.optimizers.RMSprop(lr=0.0001), metrics=['accuracy'])
checkpointer = K.callbacks.ModelCheckpoint(filepath='cifar10.h5',
     monitor="val_accuracy", verbose=1, save_best_only=True)
model.fit(trainX, trainy, batch_size=32, epochs=10, verbose=1,
     callbacks=[checkpointer], validation_data=(testX, testy),
     shuffle=True)
model.summary()
# model.save("cifar10.h5")

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/5
1562/1563 [============================>.] - ETA: 0s - loss: 0.1012 - accuracy: 0.8577
Epoch 1: val_accuracy improved from -inf to 0.89970, saving model to cifar10.h5
1563/1563 [==============================] - 144s 88ms/step - loss: 0.1011 - accuracy: 0.8577 - val_loss: 0.0534 - val_accuracy: 0.8997
Epoch 2/5
1562/1563 [============================>.] - ETA: 0s - loss: 0.0601 - accuracy: 0.8921
Epoch 2: val_accuracy improved from 0.89970 to 0.90550, saving model to cifar10.h5
1563/1563 [==============================] - 136s 87ms/step - loss: 0.0601 - accuracy: 0.8921 - val_loss: 0.0529 - val_accuracy: 0.9055
Epoch 3/5
1563/1563 [==============================] - ETA: 0s - loss: 0.0539 - accuracy: 0.9034
Epoch 3: val_accuracy improved from 0.90550 to 0.91220, saving model to cifar10.h5
1563/1563 [==============================] - 136s 87ms/step - loss: 0.0539 - accuracy: 0.9034 - val_loss: 0.0489 - val_accuracy: 0.9122
Epoch 4/5
1562/1563 [============================>.] - 

In [ ]:
image_pred=model.predict(testX)
np.argmax(image_pred,axis=1)

313/313 [==============================] - 22s 67ms/step


array([3, 8, 8, ..., 5, 0, 7])